In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
from itertools import combinations
from decimal import Decimal

pd.set_option('display.max_columns', None)

In [2]:
def caminho_absoluto(caminho_relativo_com_barras_normais):
    
    caminho_base = os.getcwd()

    caminho_absoluto = os.path.join(caminho_base, caminho_relativo_com_barras_normais)

    return caminho_absoluto

In [3]:
situacao_final = pd.read_excel(caminho_absoluto('data/tratamento_curva_abc/dados_tratados/situacao_final.xlsx')).set_index('Ordem')
local_frac = pd.read_excel(caminho_absoluto('data/analise_curva_abc/local/datasets/local_apanha_frac.xlsx'))

In [4]:
situacao_final.iloc[[0]]

,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,Dias Pedido Geral,Ativ.Ressupr.Frac,Ativ.Ressupr.Cx,Ativ.Ressupr.Geral,Média por dia frac,Média por dia cx,Média por dia geral,Estoque Armaz.,Ender.Cx.Fechada,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Fracionado,Tipo,Qtde Desvio Picking,local
Ordem,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,106011,NARIDRIN 12 HORAS FR C/30ML,A,A,A,279389.0,6507.0,930089.0,271.0,251.0,274.0,1490.0,164.0,1490.0,1030.9557,25.9243,3394.4854,5057,016-037-01-02,50,2,100,387,500,Não,29.913,Ponta De G,3236.0,pallet


In [5]:
local_frac.iloc[[0]]

,Ender.Fracionado,modulo,local
0,29.011,1,AB


In [6]:
selecao_locais = 'AA'
numero_modulos = 5

modulos = {1:[29, 28],
           2:[27, 26],
           3:[25, 24],
           4:[23, 22],
           5:[21, 20],
           6:[19, 18],
           }

selecao_local_flowrack = (local_frac['local'] == selecao_locais)
local_flowrakc_selecionado = local_frac[selecao_local_flowrack]
enderecos_local_flowrack_selecionado = local_flowrakc_selecionado['Ender.Fracionado']

def selecionar_florack(corredores):    
   selecao_flowrack_corredordes = ((situacao_final['Tipo'] == 'Flowrack') & \
                                    (situacao_final['Ender.Fracionado'].astype(str).str.startswith(str(corredores[0])) | \
                                     situacao_final['Ender.Fracionado'].astype(str).str.startswith(str(corredores[1]))
                                    )
                                  )
   flowrack_corredores = situacao_final[selecao_flowrack_corredordes]
   return flowrack_corredores

def saida_flowrack_no_modulo_pela_classe(modulo):
    flowrack_do_modulo = selecionar_florack(modulos[modulo])

    selecao_flowrack_modulo_X_classe = flowrack_do_modulo['Ender.Fracionado'].isin(enderecos_local_flowrack_selecionado)

    flowrack_modulo_X_classe = flowrack_do_modulo[selecao_flowrack_modulo_X_classe]

    saida_flowrack_modulo_classe = flowrack_modulo_X_classe['Qtde Venda Frac'].tolist()

    return saida_flowrack_modulo_classe
 
saida_dos_modulos = [saida_flowrack_no_modulo_pela_classe(1),
                    saida_flowrack_no_modulo_pela_classe(2),
                    saida_flowrack_no_modulo_pela_classe(3),
                    saida_flowrack_no_modulo_pela_classe(4),
                    saida_flowrack_no_modulo_pela_classe(5),
                    saida_flowrack_no_modulo_pela_classe(6),
                    ]

saida_dos_modulos = saida_dos_modulos[:numero_modulos]

soma_total = 0
for lista in saida_dos_modulos:
    for valor in lista:
        soma_total += valor
        
media_saida_modulo = round(soma_total / len(saida_dos_modulos), 1)

saidas_list = []
for i in range(len(saida_dos_modulos)):
    sum_lista_i = round(sum(saida_dos_modulos[i]), 1)
    saida_dif_sum_lista_i = round((sum_lista_i - media_saida_modulo), 1)
    saidas_list.append(Decimal(str(saida_dif_sum_lista_i)))

In [12]:
flowrack_do_modulo = selecionar_florack(modulos[1])

selecao_flowrack_modulo_X_classe = flowrack_do_modulo['Ender.Fracionado'].isin(enderecos_local_flowrack_selecionado)

flowrack_modulo_X_classe = flowrack_do_modulo[selecao_flowrack_modulo_X_classe]

saida_flowrack_modulo_classe = flowrack_modulo_X_classe['Qtde Venda Frac'].tolist()

enderecos_local_flowrack_selecionado.dtype

Ordem
4        True
9       False
20       True
22       True
26      False
        ...  
1208    False
1257     True
1270    False
1412    False
1771     True
Name: Ender.Fracionado, Length: 89, dtype: bool

In [8]:
saidas_list[0]

Decimal('2384.6')

In [9]:
saidas_list

[Decimal('2384.6'),
 Decimal('-217630.4'),
 Decimal('-32600.4'),
 Decimal('102903.6'),
 Decimal('144942.6')]

In [10]:
sum(saidas_list)

Decimal('0.0')

In [11]:
def encontrar_combinacao(lista, alvo):
    for i in range(1, len(lista) + 1):
        for comb in combinations(lista, i):
            if sum(comb) == alvo:
                return list(comb)
    return None

# Lista de números
modulo = saida_dos_modulos[4]

# Número alvo
total_de_saida_desejada = 217630

# Encontrar combinação
comb = encontrar_combinacao(modulo, total_de_saida_desejada)

if comb:
    print(f"Combinação encontrada: {comb}")
    i = situacao_final[situacao_final['Qtde Venda Frac'].isin(comb)]['Qtde Venda Frac'].drop_duplicates().index
    realocar = situacao_final[situacao_final.index.isin(i)]
    display(realocar)
else:
    print("Não foi possível encontrar uma combinação.")

Combinação encontrada: [98494.0, 49607.0, 23908.0, 22152.0, 14076.0, 7052.0, 1295.0, 1046.0]


,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,Dias Pedido Geral,Ativ.Ressupr.Frac,Ativ.Ressupr.Cx,Ativ.Ressupr.Geral,Média por dia frac,Média por dia cx,Média por dia geral,Estoque Armaz.,Ender.Cx.Fechada,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Fracionado,Tipo,Qtde Desvio Picking,local
Ordem,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,101175,INFRALAX C/ 15 COMP,A,A,A,98494.0,1309.0,161326.0,270.0,197.0,271.0,779.0,280.0,779.0,364.7926,6.6447,595.2989,450,014-032-02-01,30,30,48,328,332,Não,20.083,Flowrack,1983.0,apanha_a
15,102384,COMPLEXO B12 FR C/20 COMP REV,A,A,A,49607.0,147.0,64307.0,251.0,66.0,251.0,405.0,156.0,405.0,197.6375,2.2273,256.2032,166,015-025-01-01,0,12,100,395,300,Não,21.084,Flowrack,1912.0,apanha_a
43,10425,AAS 100MG C/ 30COMP,A,A,A,23908.0,55.0,28528.0,237.0,22.0,237.0,309.0,100.0,309.0,100.8776,2.5000,120.3713,0,015-027-01-02,0,10,84,75,252,Sim,20.084,Flowrack,891.0,apanha_a
48,11929,CLOR FEXOFENADINA 180MG C/ 10COMP,A,A,A,22152.0,41.0,24120.0,251.0,24.0,251.0,243.0,133.0,243.0,88.2550,1.7083,96.0956,0,014-024-02-01,0,10,48,23,192,Não,21.013,Flowrack,300.0,apanha_a
88,232429,VOLIG 8MG C/ 10COMP,A,A,A,14076.0,92.0,18492.0,242.0,13.0,242.0,213.0,150.0,213.0,58.1653,7.0769,76.4132,42,014-006-02-01,10,10,48,285,332,Não,21.073,Flowrack,507.0,apanha_a
156,11594,GLICLAZIDA CPR LIB PROL 60 MG C/ 60,A,B,A,7052.0,10.0,7612.0,124.0,4.0,124.0,57.0,36.0,57.0,56.8710,2.5000,61.3871,0,014-016-02-01,0,6,56,0,120,Não,21.053,Flowrack,282.0,apanha_a
583,130076,TORSILAX C/ 30 COMP,A,NaN,B,1295.0,NaN,1295.0,105.0,NaN,105.0,9.0,NaN,9.0,12.3333,NaN,12.3333,0,014-028-02-02,0,4,60,192,300,Não,21.064,Flowrack,72.0,apanha_a
672,14070,MELOXICAM 15MG C/ 10COMP,B,NaN,B,1046.0,NaN,1046.0,137.0,NaN,137.0,7.0,NaN,7.0,7.6350,NaN,7.6350,0,015-037-01-02,0,4,96,59,300,Não,21.082,Flowrack,34.0,apanha_a
